# Interactive Visualization

Introduction to [Bokeh](https://bokeh.pydata.org/).

In [196]:
import bokeh.plotting as bpl

# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# output to notebook
bpl.output_notebook()

# create a new plot with a title and axis labels
p = bpl.figure(title="simple line example", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness
p.line(x, y, legend="Temp.", line_width=2)

# show the results
bpl.show(p, notebook_handle=True)


Loading BokehJS ...

In [121]:
import time

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure 
output_notebook()

Loading BokehJS ...

In [4]:
N = 1000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 2
colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)]

In [5]:
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset,tap,box_select,lasso_select"

p = figure(tools=TOOLS)
p.axis.major_label_text_font_size = "18pt"
hover = HoverTool(tooltips=None, mode="vline")
p.add_tools(hover)
r = p.circle(x,y, radius=radii, 
             fill_color=colors, fill_alpha=0.6, line_color=None, 
             hover_fill_color="black", hover_fill_alpha=0.7, hover_line_color=None)

In [6]:
# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [7]:
i = 0
while True:
    i +=1 
    p.title.text = str(i)
    
    r.data_source.data['radius'] = radii * (2 + np.sin(i/5))
    
    x = r.data_source.data['x']
    y = r.data_source.data['y']
    d = np.sqrt((x-50)**2 + (y-50)**2)/100
    rand = 2 * (np.random.random(size=N) - 0.5)
    r.data_source.data['x'] = x + 2 * np.sin(d) * rand
    r.data_source.data['y'] = y + np.cos(d**2) * rand
    
    p.axis.major_label_text_color = r.data_source.data['fill_color'][int(i%N)]

    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(0.1)

KeyboardInterrupt: 

In [6]:
# Update the hover glyph propertes using the explicit handle (go hover over the plot)
r.hover_glyph.fill_color = "white"
r.hover_glyph.fill_alpha = 0.5
hover.mode = "vline"
push_notebook()

# Example with digits

In [199]:
from sklearn.datasets import load_digits
digits = load_digits()

print digits.data.shape
print digits.target.shape

SyntaxError: invalid syntax (<ipython-input-199-439cf92ea97c>, line 4)

In [200]:
from bokeh.layouts import gridplot
from bokeh.palettes import Greys256

Greys256.reverse()

i = 14

x = np.arange(10)
y = np.arange(10)
xx, yy = np.meshgrid(x, y)

f1 = figure (width=250, plot_height=250)
f1.circle(xx.flatten(), yy.flatten(), size = 10)

f2 = figure(x_range=(0, 10), y_range=(0, 10), width=150, plot_height=180, title = str(digits.target[i]))
f2.image([np.flipud(digits.data[i].reshape((8,8)))], x=0, y=0, dw=10, dh=10, palette=Greys256)

p = gridplot([[f1, f2]])
show(p)

# Hover Tools

Bokeh has a Hover Tool that allows additional information to be displayed in a popup whenever the user hovers over a specific glyph. Basic hover tool configuration amounts to providing a list of ``(name, format)`` tuples.

In [124]:
from bokeh.layouts import gridplot
from bokeh.palettes import Greys256
from bokeh.models import ColumnDataSource, HoverTool


Greys256.reverse()

i = 99

x = np.arange(10)
y = np.arange(10)
xx, yy = np.meshgrid(x, y)

source = ColumnDataSource(
        data=dict(
            x = xx.flatten(),
            y = yy.flatten(),
            label = digits.target[:xx.size]
        )
    )

f1 = figure (width=250, plot_height=250)
f1.circle('x', 'y', size = 10, source=source)
f1.add_tools(HoverTool(tooltips=[("index", "$index"), ("label:", "@label")]))

f2 = figure(x_range=(0, 10), y_range=(0, 10), width=150, plot_height=180, title = str(digits.target[i]))
f2.image([np.flipud(digits.data[i].reshape((8,8)))], x=0, y=0, dw=10, dh=10, palette=Greys256)

p = gridplot([[f1, f2]])
show(p)

# CustomJS Callbacks


In [125]:
# ESTE CODIGO NO FUNCIONA

from bokeh.layouts import gridplot, column, row 
from bokeh.palettes import Greys256
from bokeh.models import ColumnDataSource, HoverTool, CustomJS


Greys256.reverse()

i = 99

x = np.arange(10)
y = np.arange(10)
xx, yy = np.meshgrid(x, y)

N_im = xx.size
images = []
for i in range(N_im):
    images.append([np.flipud(digits.data[i].reshape((8,8)))])
    
source = ColumnDataSource(
        data=dict(
            x = xx.flatten(),
            y = yy.flatten(),
            label = digits.target[:xx.size]
        )
    )

images_src = ColumnDataSource(data=dict(images=images, index=np.arange(N_im)))
image_source = ColumnDataSource(data=dict(image=images[i]))

f1 = figure (width=250, plot_height=250)
cr = f1.circle('x', 'y', size = 10, source=source)

f2 = figure(x_range=(0, 10), y_range=(0, 10), width=150, plot_height=180, title = str(digits.target[i]))
f2.image('image', x=0, y=0, dw=10, dh=10, palette=Greys256, source = image_source)

p = gridplot([[f1, f2]])
callback = CustomJS(args=dict(dots = source, ims_src = images_src, im_src= image_source), code="""
        var selected_idx = cb_data.index['1d'].indices;
        if (selected_idx.length > 0){
            var sel_im = ims_src.data.images[selected_idx[0]];
            im_src.data= {image: [sel_im]};
            im_src.change.emit();
        }
    """)



f1.add_tools(HoverTool(tooltips=[("index", "$index"), ("label:", "@label")],
                        callback=callback, renderers=[cr]))


show(p)

In [126]:
from bokeh.layouts import gridplot, column, row 
from bokeh.palettes import Greys256
from bokeh.models import ColumnDataSource, HoverTool, CustomJS


Greys256.reverse()

i_sel = 99

x = np.arange(10)
y = np.arange(10)
xx, yy = np.meshgrid(x, y)

N_im = xx.size
images = []
for i in range(N_im):
    images.append(np.flipud(digits.data[i].reshape((8,8)))/16.)
print digits.data[i].max()
source = ColumnDataSource(
        data=dict(
            x = xx.flatten(),
            y = yy.flatten(),
            label = digits.target[:xx.size]
        )
    )

images_src = ColumnDataSource(data=dict(images=images, index=np.arange(N_im)))

x_sq = np.arange(8)+0.5
y_sq = np.arange(8)+0.5
xx_sq, yy_sq = np.meshgrid(x_sq, y_sq)
xx_f = xx_sq.flatten()
yy_f = yy_sq.flatten()
image_source = ColumnDataSource(
                data=dict(
                    x = xx_f,
                    y = yy_f,
                    image=images[i_sel].flatten()
                )
            )

f1 = figure (width=250, plot_height=250)
cr = f1.circle('x', 'y', size = 10, source=source)


f2 = figure(x_range=(0, 8), y_range=(0, 8), width=150, plot_height=180, title = str(digits.target[i]))
#f2.square(x=xx_f, y=yy_f, fill_alpha= images[i][0].flatten(), size=16, fill_color = "black", line_color = "black")
f2.square(x='x', y='y', fill_alpha= 'image', size=16, fill_color = "black", line_color = "black", source = image_source)

p = gridplot([[f1, f2]])
callback = CustomJS(args=dict(dots = source, ims_src = images_src, im_src= image_source), code="""
        var selected_idx = cb_data.index['1d'].indices;
        if (selected_idx.length > 0){
            var sel_im = ims_src.data.images[selected_idx[0]];
            im_src.data['image']= sel_im;
            im_src.change.emit();
        }
    """)



f1.add_tools(HoverTool(tooltips=[("index", "$index"), ("label:", "@label")],
                        callback=callback, renderers=[cr]))


show(p)

16.0


## Convolutional autoencoder


In [127]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(8, 8, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x) 
x = Conv2D(4, (2, 2), activation='relu', padding='same')(x) 
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(2, (1, 1), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (1, 1, 2)

x = Conv2D(2, (1, 1), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x) # size 2 after this
x = Conv2D(4, (2, 2), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x) 
x = Conv2D(8, (2, 2), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [128]:
images = []
for i in range(len(digits.target)):
    images.append(np.flipud(digits.data[i].reshape((8,8)))/16.)
x_train = np.reshape(np.array(images), (len(images), 8, 8, 1))

In [129]:
from keras.callbacks import TensorBoard

history = autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_split=0.2)

Train on 1437 samples, validate on 360 samples
Epoch 1/50
1437/1437 [==============================] - 0s - loss: 0.6918 - val_loss: 0.6898
Epoch 2/50
1437/1437 [==============================] - 0s - loss: 0.6870 - val_loss: 0.6815
Epoch 3/50
1437/1437 [==============================] - 0s - loss: 0.6720 - val_loss: 0.6543
Epoch 4/50
1437/1437 [==============================] - 0s - loss: 0.6428 - val_loss: 0.6333
Epoch 5/50
1437/1437 [==============================] - 0s - loss: 0.6262 - val_loss: 0.6136
Epoch 6/50
1437/1437 [==============================] - 0s - loss: 0.6027 - val_loss: 0.5810
Epoch 7/50
1437/1437 [==============================] - 0s - loss: 0.5660 - val_loss: 0.5396
Epoch 8/50
1437/1437 [==============================] - 0s - loss: 0.5339 - val_loss: 0.5158
Epoch 9/50
1437/1437 [==============================] - 0s - loss: 0.5172 - val_loss: 0.5047
Epoch 10/50
1437/1437 [==============================] - 0s - loss: 0.5089 - val_loss: 0.4987
Epoch 11/50
1437/1437 

In [130]:
p = bpl.figure(title="learning history", x_axis_label='minibatch', y_axis_label='loss')

# add a line renderer with legend and line thickness
p.line(np.arange(len(history.history['loss'])), history.history['loss'], legend="train.", line_width=2)
p.line(np.arange(len(history.history['val_loss'])), history.history['val_loss'], legend="validation.", line_width=2, line_color="orange")

# show the results
bpl.show(p, notebook_handle=True)

In [131]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

encoded_imgs = encoder.predict(x_train)
print encoded_imgs.shape
encoded_imgs = encoded_imgs.reshape(encoded_imgs.shape[0], 2)
print encoded_imgs.shape


(1797, 1, 1, 2)
(1797, 2)


In [133]:
from bokeh.layouts import gridplot, column, row 
from bokeh.palettes import Greys256
from bokeh.models import ColumnDataSource, HoverTool, CustomJS


Greys256.reverse()

i_sel = 99

xx, yy = encoded_imgs[:100, 0], encoded_imgs[:100, 1]

N_im = xx.size
images = []
for i in range(N_im):
    images.append(np.flipud(digits.data[i].reshape((8,8)))/16.)
print digits.data[i].max()
source = ColumnDataSource(
        data=dict(
            x = xx.flatten(),
            y = yy.flatten(),
            label = digits.target[:xx.size]
        )
    )

images_src = ColumnDataSource(data=dict(images=images, index=np.arange(N_im)))

x_sq = np.arange(8)+0.5
y_sq = np.arange(8)+0.5
xx_sq, yy_sq = np.meshgrid(x_sq, y_sq)
xx_f = xx_sq.flatten()
yy_f = yy_sq.flatten()
image_source = ColumnDataSource(
                data=dict(
                    x = xx_f,
                    y = yy_f,
                    image=images[i_sel].flatten()
                )
            )

f1 = figure (width=250, plot_height=250, y_axis_type="log")
cr = f1.circle('x', 'y', size = 4, source=source)


f2 = figure(x_range=(0, 8), y_range=(0, 8), width=150, plot_height=180, title = str(digits.target[i]))
#f2.square(x=xx_f, y=yy_f, fill_alpha= images[i][0].flatten(), size=16, fill_color = "black", line_color = "black")
f2.square(x='x', y='y', fill_alpha= 'image', size=16, fill_color = "black", line_color = "black", source = image_source)

p = gridplot([[f1, f2]])
callback = CustomJS(args=dict(dots = source, ims_src = images_src, im_src= image_source), code="""
        var selected_idx = cb_data.index['1d'].indices;
        if (selected_idx.length > 0){
            var sel_im = ims_src.data.images[selected_idx[0]];
            im_src.data['image']= sel_im;
            im_src.change.emit();
        }
    """)



f1.add_tools(HoverTool(tooltips=[("index", "$index"), ("label:", "@label")],
                        callback=callback, renderers=[cr]))


show(p)

16.0


## Variational autoencoder (VAE)

Variational autoencoders are a slightly more modern and interesting take on autoencoding. This example was modified from the one found [here](https://blog.keras.io/building-autoencoders-in-keras.html).

What is a variational autoencoder, you ask? It's a type of autoencoder with added constraints on the encoded representations being learned. More precisely, it is an autoencoder that learns a latent variable model for its input data. So instead of letting your neural network learn an arbitrary function, you are learning the parameters of a probability distribution modeling your data. If you sample points from this distribution, you can generate new input data samples: a VAE is a "generative model".

How does a variational autoencoder work?

First, an encoder network turns the input samples x into two parameters in a latent space, which we will note z_mean and z_log_sigma. Then, we randomly sample similar points z from the latent normal distribution that is assumed to generate the data, via z = z_mean + exp(z_log_sigma) * epsilon, where epsilon is a random normal tensor. Finally, a decoder network maps these latent space points back to the original input data.

The parameters of the model are trained via two loss functions: a reconstruction loss forcing the decoded samples to match the initial inputs (just like in our previous autoencoders), and the KL divergence between the learned latent distribution and the prior distribution, acting as a regularization term. You could actually get rid of this latter term entirely, although it does help in learning well-formed latent spaces and reducing overfitting to the training data.

In [146]:
from keras import backend as K
from keras.layers import Lambda, Input, Dense

# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


In [149]:
intermediate_dim = 32
latent_dim = 2

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=(64, ), name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
encoder_input (InputLayer)       (None, 64)            0                                            
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 32)            2080        encoder_input[0][0]              
____________________________________________________________________________________________________
z_mean (Dense)                   (None, 2)             66          dense_5[0][0]                    
____________________________________________________________________________________________________
z_log_var (Dense)                (None, 2)             66          dense_5[0][0]                    
___________________________________________________________________________________________

In [150]:
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

What we've done so far allows us to instantiate 3 models:

an end-to-end autoencoder mapping inputs to reconstructions
an encoder mapping inputs to the latent space
a generator that can take points on the latent space and will output the corresponding reconstructed samples.

In [157]:
from keras.losses import binary_crossentropy

reconstruction_loss = binary_crossentropy(inputs, outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.compile(optimizer='adam', loss='binary_crossentropy')
vae.add_loss(vae_loss)
vae.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 64)                0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 2212      
_________________________________________________________________
decoder (Model)              (None, 64)                2208      
Total params: 4,420
Trainable params: 4,420
Non-trainable params: 0
_________________________________________________________________


In [168]:
images = []
for i in range(len(digits.target)):
    images.append(digits.data[i]/16.)
x_train = np.reshape(np.array(images), (len(images), 64))
print x_train.shape

(1797, 64)


In [169]:
history = vae.fit(x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_split=0.2)

AttributeError: 'NoneType' object has no attribute 'shape'

In [166]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

image_size = x_train.shape[1]
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [167]:
print x_train.shape


(60000, 784)


In [4]:
'''Example of VAE on MNIST dataset using MLP
The VAE has a modular design. The encoder, decoder and VAE
are 3 models that share weights. After training the VAE model,
the encoder can be used to  generate latent vectors.
The decoder can be used to generate MNIST digits by sampling the
latent vector from a Gaussian distribution with mean=0 and std=1.
# Reference
[1] Kingma, Diederik P., and Max Welling.
"Auto-encoding variational bayes."
https://arxiv.org/abs/1312.6114
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import os


# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as function of 2-dim latent vector
    # Arguments:
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()


# MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

image_size = x_train.shape[1]
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (original_dim, )
intermediate_dim = 512
batch_size = 128
latent_dim = 2
epochs = 50

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

if __name__ == '__main__':
    mse = False
    models = (encoder, decoder)
    data = (x_test, y_test)

    # VAE loss = mse_loss or xent_loss + kl_loss
    if mse:
        reconstruction_loss = mse(inputs, outputs)
    else:
        reconstruction_loss = binary_crossentropy(inputs,
                                                  outputs)

    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.compile(optimizer='adam', loss = "binary_crossentropy")
    vae.add_loss(vae_loss)
    vae.summary()



    # train the autoencoder
    vae.fit(x_train,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(x_test, None))

    plot_results(models,
                 data,
                 batch_size=batch_size,
                 model_name="vae_mlp")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
encoder_input (InputLayer)       (None, 784)           0                                            
____________________________________________________________________________________________________
dense_11 (Dense)                 (None, 512)           401920      encoder_input[0][0]              
____________________________________________________________________________________________________
z_mean (Dense)                   (None, 2)             1026        dense_11[0][0]                   
____________________________________________________________________________________________________
z_log_var (Dense)                (None, 2)             1026        dense_11[0][0]                   
___________________________________________________________________________________________

AttributeError: 'NoneType' object has no attribute 'shape'

In [172]:
%tb

SystemExit: 2

In [180]:
from keras.layers import Input, Dense, Lambda, Layer, Add, Multiply

class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs


In [181]:
x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(x)

z_mu = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])
z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_var)

eps = Input(tensor=K.random_normal(shape=(K.shape(x)[0],
                                          latent_dim)))
z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])

decoder = Sequential([
    Dense(intermediate_dim, input_dim=latent_dim, activation='relu'),
    Dense(original_dim, activation='sigmoid')
])

x_pred = decoder(z)

IndexError: list index out of range

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras import backend as K

from keras.layers import Input, Dense, Lambda, Layer, Add, Multiply
from keras.models import Model, Sequential
from keras.datasets import mnist


original_dim = 784
intermediate_dim = 256
latent_dim = 2
batch_size = 100
epochs = 50
epsilon_std = 1.0


def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """

    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)


class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs


decoder = Sequential([
    Dense(intermediate_dim, input_dim=latent_dim, activation='relu'),
    Dense(original_dim, activation='sigmoid')
])

x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(x)

z_mu = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])
z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_var)

eps = Input(tensor=K.random_normal(stddev=epsilon_std,
                                   shape=(K.shape(x)[0], latent_dim)))
z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])

x_pred = decoder(z)

vae = Model(inputs=[x, eps], outputs=x_pred)
vae.compile(optimizer='rmsprop', loss=nll)

# train the VAE on MNIST digits
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, original_dim) / 255.
x_test = x_test.reshape(-1, original_dim) / 255.

vae.fit(x_train,
        x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test))

encoder = Model(x, z_mu)

# display a 2D plot of the digit classes in the latent space
z_test = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(z_test[:, 0], z_test[:, 1], c=y_test,
            alpha=.4, s=3**2, cmap='viridis')
plt.colorbar()
plt.show()

# display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28

# linearly spaced coordinates on the unit square were transformed
# through the inverse CDF (ppf) of the Gaussian to produce values
# of the latent variables z, since the prior of the latent space
# is Gaussian
u_grid = np.dstack(np.meshgrid(np.linspace(0.05, 0.95, n),
                               np.linspace(0.05, 0.95, n)))
z_grid = norm.ppf(u_grid)
x_decoded = decoder.predict(z_grid.reshape(n*n, 2))
x_decoded = x_decoded.reshape(n, n, digit_size, digit_size)

plt.figure(figsize=(10, 10))
plt.imshow(np.block(list(map(list, x_decoded))), cmap='gray')
plt.show()

IndexError: list index out of range

In [1]:

import keras;
import keras.backend

print (keras.backend.backend())
if keras.backend.backend() != 'theano':
    raise BaseException("This script uses other backend")
else:
    keras.backend.set_image_dim_ordering('th')
    print("Backend ok")

Using TensorFlow backend.


tensorflow


BaseException: This script uses other backend